# Introduction


??????????????????????????????

# Libraries

In [1]:
import pandas as pd
import numpy as np

! python -m spacy download en_core_web_md
import spacy
spacy.prefer_gpu()

import plotly.express as px

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import preprocessing
from keras.preprocessing.sequence import pad_sequences

import warnings; warnings.simplefilter('ignore')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


Using TensorFlow backend.


# Dataset

In [2]:
df = pd.read_csv('/content/drive/My Drive/ML Projects/Womens Clothing E-Commerce/Womens Clothing E-Commerce Reviews.csv')
df

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


## Useful Data

For this notebook only the review and the recommended IDN will be used


In [3]:
df = df[~df['Review Text'].isna()].reset_index()
df

,index,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...,...
22636,23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
22637,23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
22638,23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
22639,23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [4]:
df = df[['Review Text', 'Recommended IND']]
df

,Review Text,Recommended IND
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1
...,...,...
22636,I was very happy to snag this dress at such a ...,1
22637,"It reminds me of maternity clothes. soft, stre...",1
22638,"This fit well, but the top was very see throug...",0
22639,I bought this dress for a wedding i have this ...,1


# Spliting Data

In [5]:
train, test = train_test_split(df,test_size = 0.3,random_state=7)
test, val = train_test_split(test,test_size=0.5,random_state=7)

train.reset_index(inplace = True,drop = True)
val.reset_index(inplace = True,drop = True)
test.reset_index(inplace = True,drop = True)

# Creating Vocabulary

Insted of use a BagOfWords approach ....


In [6]:
vectorizer = CountVectorizer()
vectorizer.fit(train['Review Text'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [7]:
vectorizer.vocabulary_

{'coveted': 2733,
 'this': 10725,
 'blouse': 1490,
 'for': 4437,
 'weeks': 11701,
 'and': 816,
 'finally': 4213,
 'pressed': 8118,
 'go': 4762,
 'during': 3556,
 'the': 10663,
 'recent': 8479,
 'sale': 9010,
 'it': 5688,
 'is': 5677,
 'truly': 11040,
 'lovely': 6328,
 'textured': 10650,
 'silk': 9482,
 'its': 5705,
 'lining': 6187,
 'combine': 2400,
 'to': 10844,
 'give': 4721,
 'weight': 11707,
 'needs': 7038,
 'flow': 4363,
 'perfectly': 7699,
 'when': 11750,
 'untucked': 11336,
 'shaped': 9291,
 'fuller': 4563,
 'at': 1046,
 'hem': 5088,
 'tucked': 11068,
 'neat': 7004,
 'as': 1000,
 'pin': 7820,
 'colors': 2378,
 'are': 956,
 'almost': 753,
 'metallic': 6666,
 'looks': 6266,
 'if': 5370,
 'flowers': 4367,
 'have': 5016,
 'been': 1302,
 'painted': 7518,
 'on': 7285,
 'an': 810,
 'elegant': 3660,
 'casual': 1989,
 'look': 6260,
 'with': 11869,
 'white': 11780,
 'jeans': 5740,
 'heels': 5072,
 'or': 7342,
 've': 11438,
 'dressed': 3487,
 'up': 11351,
 'black': 1435,
 'pencil': 7671,
 

Number of words 

In [8]:
max(vectorizer.vocabulary_.values())

12086

## Vocabulary redunction

There are to many words in the vocabulary and the text has no treatment.

Lets try to solve this problems


In [9]:
def flatten(lst):
    new_lst = []
    flatten_helper(lst, new_lst)
    return new_lst
 
def flatten_helper(lst, new_lst):
    for element in lst:
        if isinstance(element, list):
            flatten_helper(element, new_lst)
        else:
            new_lst.append(element)

def RemoveSmallWords(Series):
    t = Series.str.split(expand=True).stack()
    return t.loc[t.str.len() >= 4].groupby(level=0).apply(' '.join)

def TextPreProcessing (df,text_column):
  df.loc[df[text_column].isna(),text_column] = '-EMPTY-'
  # Creating Corpus with Spacy pipe.
  nlp = spacy.load('en_core_web_md')
  corpus = list(nlp.pipe(df[text_column]))
  a_lemmas = []
  for i in range(len(corpus)):
    try:
      #creating name entity recognition list for the especific corpus
      ents = [ent.text.split() for ent in corpus[i].ents]
      ents = flatten(ents)    
      # Tokenization with lemmatizer
      lemmas = [token.lemma_ for token in corpus[i]]
      # Removing non-alphabetic characters
      a_lemmas.append(pd.Series([lemma for lemma in lemmas if (lemma.isalpha() and nlp.vocab[lemma].is_stop==False)])) 
      a_lemmas[i] = a_lemmas[i][~a_lemmas[i].isin(pd.Series(ents))]
      a_lemmas[i] = RemoveSmallWords(a_lemmas[i])
      a_lemmas[i].reset_index(inplace = True, drop = True)
    except:
      a_lemmas[i] = '-EMPTY-'
      a_lemmas[i].reset_index(inplace = True, drop = True)
    a_lemmas[i] = ' '.join(a_lemmas[i])
  df[text_column] = a_lemmas
  return df

In [10]:
train_review = TextPreProcessing (df = train,text_column = 'Review Text')
val_review = TextPreProcessing (df = val,text_column = 'Review Text')
test_review = TextPreProcessing (df = test,text_column = 'Review Text')


train_review_copy = train_review.copy()
val_review_copy = val_review.copy()
test_review_copy = test_review.copy()

In [11]:
# train_review = train_review_copy.copy()
# val_review = val_review_copy.copy()
# test_review = test_review_copy.copy()

Dicionario de pavras, acgora limpo

In [12]:
vectorizer = CountVectorizer()
vectorizer.fit(train_review['Review Text'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

Escrever sobre a reducao de palavras

In [13]:
max(vectorizer.vocabulary_.values())

7677

# Word to Number

Explicar a necessidade de trocar as pavras por numeros

Frase com maior e menor numero de plavras

Sera necessario tokenizar novamente as frases, eh um retrabalho, e poderia ter sido feito na funcao de pre processamento de texto.

Para isso bastave ter comentado a linha: `a_lemmas[i] = ' '.join(a_lemmas[i])`
 

In [14]:
def tokenize_sentence(df,sentence_column):
  nlp = spacy.load('en_core_web_md')
  df[sentence_column] = df[sentence_column].apply(nlp.tokenizer)
  df['sentence_size'] = 0
  for i in range(df.shape[0]):
    df[sentence_column][i] = list(df[sentence_column][i])
  df['sentence_size'] = df[sentence_column].apply(len)
  return df

In [15]:
train_review = tokenize_sentence(train_review,'Review Text')
val_review = tokenize_sentence(val_review,'Review Text')
test_review = tokenize_sentence(test_review,'Review Text')

Distribution of quantity of words in sentences

In [16]:
fig = px.histogram(train_review, x="sentence_size")
fig.show()

## Data Cleaning

Quantile approach will be implemented, to remove sentence with 

In [17]:
train_review = train_review[(train_review['sentence_size'] > train_review['sentence_size'].quantile(0.05)) | (train_review['sentence_size'] > train_review['sentence_size'].quantile(0.95))]
train_review.reset_index(inplace = True, drop = True)

In [18]:
print('Greater number of words in a sentence = {}'.format(train_review['sentence_size'].max()))
print('Least number of words in a sentence = {}'.format(train_review['sentence_size'].min()))

Greater number of words in a sentence = 46
Least number of words in a sentence = 7


Word to number with CountVectorizer

In [19]:
def wordToNumber(df,list_words_column,vectorizer):
  list_ = []
  for i in range(df.shape[0]):
    for word in df[list_words_column][i]:
      if str(word) in vectorizer.vocabulary_.keys():
          list_.append( vectorizer.vocabulary_[str(word)])
    df[list_words_column][i] = list_
    list_ = []
  return df

In [20]:
train_review = wordToNumber(train_review,'Review Text',vectorizer)
val_review = wordToNumber(val_review,'Review Text',vectorizer)
test_review = wordToNumber(test_review,'Review Text',vectorizer)

In [21]:
train_review

,Review Text,Recommended IND,sentence_size
0,"[1497, 645, 7464, 2483, 5041, 5295, 5650, 6982...",1,37
1,"[471, 3834, 2485, 2539, 4861, 7511, 876, 5897,...",1,32
2,"[11, 2485, 4028, 2014, 5048, 5917, 4779, 1267,...",1,33
3,"[654, 5567, 3872, 1728, 6303, 2800, 1373, 5122...",1,29
4,"[7405, 7586, 4517, 3514, 1239, 7405, 5784, 100...",1,23
...,...,...,...
14865,"[5490, 6372, 5635, 2930, 2091, 7384, 3325, 276...",1,21
14866,"[6848, 6372, 11, 3755, 5057, 5093, 1239, 5622,...",1,29
14867,"[3872, 5842, 2587, 2014, 5789, 1868, 6295, 201...",0,9
14868,"[3806, 654, 3531, 2344, 4672, 6466, 2354, 2242...",1,28


## Differents sizes

It's possible to observe that the sentences have differents sizes. it's necessary to have all the sentences with the same size to use a RNN to predict with a costumer will recommendet or not.

Using padding to have sentences with same size

In [22]:
x_train =  pad_sequences( train_review['Review Text'].values, maxlen= train_review['sentence_size'].max(), padding = 'pre', truncating='post')
y_train =  train_review['Recommended IND'].values

x_val =  pad_sequences( val_review['Review Text'].values, maxlen= val_review['sentence_size'].max(), padding = 'pre', truncating='post')
y_val =  val_review['Recommended IND'].values

x_test =  pad_sequences( test_review['Review Text'].values, maxlen= test_review['sentence_size'].max(), padding = 'pre', truncating='post')
y_test =  test_review['Recommended IND'].values

In [23]:
x_train

array([[   0,    0,    0, ...,  604, 4730, 6028],
       [   0,    0,    0, ..., 1196, 6367, 3872],
       [   0,    0,    0, ..., 2014, 1841, 4697],
       ...,
       [   0,    0,    0, ..., 6295, 2014, 5311],
       [   0,    0,    0, ..., 3531, 1728,  920],
       [   0,    0,    0, ..., 6848, 3872, 2014]], dtype=int32)

In [26]:
del train_review,val_review,test_review,vectorizer,train_review_copy,val_review_copy,test_review_copy,train,val,test

# RNN Models